In [1]:
!pip3 install gpt-index
!pip3 install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from gpt_index import LangchainEmbedding

# 埋め込みモデルの準備
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(
    model_name="oshizo/sbert-jsnli-luke-japanese-base-lite"
))

In [4]:
from gpt_index import GPTSimpleVectorIndex, SimpleDirectoryReader, PromptHelper

# インデックスの作成
documents = SimpleDirectoryReader('data').load_data()

In [5]:
# インデックスの生成
index = GPTSimpleVectorIndex(
    documents=documents,
    prompt_helper=PromptHelper(
        max_input_size=4000,  # LLM入力の最大トークン数
        num_output=256,  # LLM出力のトークン数
        chunk_size_limit=500,  # チャンクのトークン数
        max_chunk_overlap=0,  # チャンクオーバーラップの最大トークン数
        separator="。"  # セパレータ
    ),
    embed_model=embed_model,  # 埋め込みモデル
    verbose=True
)

> Adding chunk: あらすじ
結束バンド
後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代に...
> Adding chunk: ひとりは虹夏、山田リョウらと共に「結束バンド」として活動を始めるが、友達は欲しいけどコミュ症独...
> Adding chunk: きくりのお陰でライブチケットのノルマもこなし、結束バンドはライブに臨むのだった。

文化祭ライ...
> Adding chunk: その演奏をみんなから拍手喝采されるひとりだったが、コミュ症のひとりはその晴れ舞台に混乱し、観客...
> Adding chunk: Harry Potter is a series of seven fantasy novel...
> [build_index_from_documents] Total LLM token usage: 0 tokens
> [build_index_from_documents] Total embedding token usage: 2007 tokens


In [6]:
print(index.query(
    "後藤ひとりはどういう性格？", 
    verbose=True
))

> Top 1 nodes:
> [Node 5c12161e-96cb-42e6-816a-245d63a614f9] [Similarity score:                     0.434401] あらすじ
結束バンド
後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代にテレビのインタビューを見て、陰キャでもバンドを組んでいれば人気者になれると聞き、ギターの練習を始める...
> Searching in chunk: あらすじ
結束バンド
後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代に...
> Initial response: 
後藤ひとりは陰キャで、友達を作ることができず、現実では変わらず孤独な性格であると考えられます。
> [query] Total LLM token usage: 595 tokens
> [query] Total embedding token usage: 22 tokens

後藤ひとりは陰キャで、友達を作ることができず、現実では変わらず孤独な性格であると考えられます。


In [7]:
print(index.query(
    "ハリーの性格は？", 
    verbose=True
))

> Top 1 nodes:
> [Node f7b1b4e3-84e2-466a-bf82-a67647b98538] [Similarity score:                     0.4788] Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The nov...
> Searching in chunk: Harry Potter is a series of seven fantasy novel...
> Initial response: 
Harry Potter is portrayed as a brave, loyal, and determined character. He is willing to put himself in danger to protect his friends and family, and is often willing to take risks to achieve his goals. He is also shown to be kind and compassionate, and is willing to forgive those who have wronged him.
> [query] Total LLM token usage: 341 tokens
> [query] Total embedding token usage: 12 tokens

Harry Potter is portrayed as a brave, loyal, and determined character. He is willing to put himself in danger to protect his friends and family, and is often willing to take risks to achieve his goals. He is also shown to be kind and compassionate, and is willing to forgive those who have wronged hi

In [8]:
print(index.query(
    "who is the guitar hero？", 
    verbose=True,
    similarity_top_k=2
))

> Top 2 nodes:
> [Node f7b1b4e3-84e2-466a-bf82-a67647b98538] [Similarity score:                     0.713711] Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The nov...
> [Node 78548977-9cf7-491b-8f85-3e8f4ba79e3e] [Similarity score:                     0.23417] その演奏をみんなから拍手喝采されるひとりだったが、コミュ症のひとりはその晴れ舞台に混乱し、観客席にダイブするという暴挙に出てしまう。ひとまずライブは成功を収めるものの、ひとりの奇行により、結束バ...
> Searching in chunk: Harry Potter is a series of seven fantasy novel...
> Searching in chunk: その演奏をみんなから拍手喝采されるひとりだったが、コミュ症のひとりはその晴れ舞台に混乱し、観客...
> Initial response: 
The guitar hero is not mentioned in the context information, so the answer is unknown.
> Refine context: その演奏をみんなから拍手喝采されるひとりだったが、コミュ症のひとりはその晴れ舞台に混乱し、観客...
> Refined response: 
The guitar hero is not mentioned in the context information, but it is implied that one of the members of the band is a guitar hero.
> [query] Total LLM token usage: 789 tokens
> [query] Total embedding token usage: 8 tokens

The guitar hero is not m